In [30]:
import difflib
import numpy as np
import os
import pandas as pd
from scipy.spatial import distance_matrix
from sklearn.impute import KNNImputer
import warnings

warnings.simplefilter("ignore", pd.errors.SettingWithCopyError)
pd.set_option('display.max_columns', None)

# Rainfall

In [2]:
df_rainfall = pd.read_csv("../datasets/checkpoints/rainfall.csv")

display(df_rainfall.info())
display(df_rainfall.describe())
display(df_rainfall)
df_rainfall.isna().mean()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80651784 entries, 0 to 80651783
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Hour           int64  
 1   Precipitation  float64
 2   Date           object 
 3   Lat            float64
 4   Long           float64
dtypes: float64(3), int64(1), object(1)
memory usage: 3.0+ GB


None

,Hour,Precipitation,Lat,Long
count,8.065178e+07,8.061156e+07,8.065178e+07,8.065178e+07
mean,1.150000e+01,2.602369e-01,5.569723e+00,-7.461878e+01
std,6.922187e+00,1.107324e+00,2.569344e+00,1.690394e+00
min,0.000000e+00,0.000000e+00,-4.203165e+00,-8.171853e+01
25%,5.750000e+00,0.000000e+00,4.263082e+00,-7.574878e+01
50%,1.150000e+01,0.000000e+00,5.516347e+00,-7.474094e+01
75%,1.725000e+01,2.000000e-01,6.849736e+00,-7.342148e+01
max,2.300000e+01,2.935000e+02,1.338142e+01,-6.704646e+01


,Hour,Precipitation,Date,Lat,Long
0,0,0.1,2010-01-01,-1.747145,-73.209101
1,1,0.1,2010-01-01,-1.747145,-73.209101
2,2,0.1,2010-01-01,-1.747145,-73.209101
3,3,0.1,2010-01-01,-1.747145,-73.209101
4,4,0.1,2010-01-01,-1.747145,-73.209101
...,...,...,...,...,...
80651779,19,NaN,2018-12-31,2.892506,-72.133175
80651780,20,0.0,2018-12-31,2.892506,-72.133175
80651781,21,0.0,2018-12-31,2.892506,-72.133175
80651782,22,0.0,2018-12-31,2.892506,-72.133175


Hour             0.000000
Precipitation    0.000499
Date             0.000000
Lat              0.000000
Long             0.000000
dtype: float64

In [3]:
df_rainfall["Precipitation"] = df_rainfall["Precipitation"].fillna(0)

df_rainfall["Precipitation"].describe()

count    8.065178e+07
mean     2.601072e-01
std      1.107063e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.000000e-01
max      2.935000e+02
Name: Precipitation, dtype: float64

In [4]:
df_rainfall_total = (
    df_rainfall
        .groupby(["Date", "Lat", "Long"])["Precipitation"]
        .sum()
        .reset_index(name = "Total Rainfall")
)
df_rainfall_duration = (
    df_rainfall
        .groupby(["Date", "Lat", "Long"])["Precipitation"]
        .apply(lambda x: (x > 0).sum())
        .reset_index(name = "Rainfall Duration")
)
df_rainfall2 = pd.merge(
    df_rainfall_total,
    df_rainfall_duration,
    on = ["Date", "Lat", "Long"]
)

display(df_rainfall2.info())
display(df_rainfall2.describe())
display(df_rainfall2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3360491 entries, 0 to 3360490
Data columns (total 5 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Date               object 
 1   Lat                float64
 2   Long               float64
 3   Total Rainfall     float64
 4   Rainfall Duration  int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 128.2+ MB


None

,Lat,Long,Total Rainfall,Rainfall Duration
count,3.360491e+06,3.360491e+06,3.360491e+06,3.360491e+06
mean,5.569723e+00,-7.461878e+01,6.242572e+00,1.044415e+01
std,2.569344e+00,1.690394e+00,1.000245e+01,8.417210e+00
min,-4.203165e+00,-8.171853e+01,0.000000e+00,0.000000e+00
25%,4.263082e+00,-7.574878e+01,4.000000e-01,2.000000e+00
50%,5.516347e+00,-7.474094e+01,2.800000e+00,1.000000e+01
75%,6.849736e+00,-7.342148e+01,8.300000e+00,1.700000e+01
max,1.338142e+01,-6.704646e+01,3.723000e+02,2.400000e+01


,Date,Lat,Long,Total Rainfall,Rainfall Duration
0,2010-01-01,-4.203165,-69.935907,0.4,4
1,2010-01-01,-3.788867,-70.355642,0.5,5
2,2010-01-01,-2.458989,-69.859741,0.3,3
3,2010-01-01,-1.747145,-73.209101,0.5,5
4,2010-01-01,-1.378972,-72.804280,0.0,0
...,...,...,...,...,...
3360486,2018-12-31,11.240355,-74.211023,0.0,0
3360487,2018-12-31,11.380054,-72.242246,0.0,0
3360488,2018-12-31,11.538415,-72.916784,0.0,0
3360489,2018-12-31,11.576376,-72.567804,0.0,0


In [5]:
df_rainfall = df_rainfall2.copy()

del df_rainfall_total
del df_rainfall_duration
del df_rainfall2

# Forest Data

In [6]:
df_forest = pd.read_csv("../datasets/checkpoints/forest_loss.csv")

df_forest.rename(
    {
        "subnational1": "Department",
        "subnational2": "Municipality"
    },
    axis = 1,
    inplace = True
)

display(df_forest.info())
display(df_forest.describe())
df_forest

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187440 entries, 0 to 187439
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   country            187440 non-null  object 
 1   Department         187440 non-null  object 
 2   Municipality       187440 non-null  object 
 3   Lat                187440 non-null  float64
 4   Long               187440 non-null  float64
 5   elevation          187440 non-null  float64
 6   threshold          187440 non-null  int64  
 7   Year               187440 non-null  int64  
 8   gain_2000-2020_ha  187440 non-null  int64  
 9   extent_2010_ha     187440 non-null  int64  
 10  area_ha            187440 non-null  int64  
 11  extent_2000_ha     187440 non-null  int64  
 12  forest_loss_ha     187440 non-null  int64  
dtypes: float64(3), int64(7), object(3)
memory usage: 18.6+ MB


None

,Lat,Long,elevation,threshold,Year,gain_2000-2020_ha,extent_2010_ha,area_ha,extent_2000_ha,forest_loss_ha
count,187440.000000,187440.000000,187440.000000,187440.000000,187440.000000,187440.000000,1.874400e+05,1.874400e+05,1.874400e+05,187440.000000
mean,5.550268,-74.630470,1189.064659,28.125000,2011.500000,1018.963380,7.952658e+04,1.067379e+05,7.975576e+04,220.545657
std,2.571924,1.703351,932.783699,22.491378,6.344306,2570.691511,2.758679e+05,3.104969e+05,2.770098e+05,1007.345449
min,-4.203165,-81.718525,0.000000,0.000000,2001.000000,0.000000,0.000000e+00,9.300000e+01,0.000000e+00,0.000000
25%,4.216550,-75.761865,223.823059,13.750000,2006.000000,75.000000,6.935750e+03,1.362200e+04,6.929750e+03,7.000000
50%,5.503838,-74.755201,1191.455566,22.500000,2011.500000,209.000000,1.757350e+04,2.882300e+04,1.744900e+04,31.000000
75%,6.854309,-73.418021,1909.895264,35.000000,2017.000000,653.000000,4.524025e+04,7.152900e+04,4.536350e+04,123.000000
max,13.381425,-67.046459,3670.898682,75.000000,2022.000000,38765.000000,4.967616e+06,4.967616e+06,4.967616e+06,31826.000000


,country,Department,Municipality,Lat,Long,elevation,threshold,Year,gain_2000-2020_ha,extent_2010_ha,area_ha,extent_2000_ha,forest_loss_ha
0,Colombia,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,0,2001,773,1027642,1027642,1027642,212
1,Colombia,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,0,2002,773,1027642,1027642,1027642,182
2,Colombia,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,0,2003,773,1027642,1027642,1027642,88
3,Colombia,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,0,2004,773,1027642,1027642,1027642,169
4,Colombia,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,0,2005,773,1027642,1027642,1027642,147
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187435,Colombia,Vichada,Santa Rosalía,5.135381,-70.864961,116.421806,75,2018,6493,28283,432087,30129,23
187436,Colombia,Vichada,Santa Rosalía,5.135381,-70.864961,116.421806,75,2019,6493,28283,432087,30129,24
187437,Colombia,Vichada,Santa Rosalía,5.135381,-70.864961,116.421806,75,2020,6493,28283,432087,30129,38
187438,Colombia,Vichada,Santa Rosalía,5.135381,-70.864961,116.421806,75,2021,6493,28283,432087,30129,67


In [7]:
df_forest = df_forest.loc[
    (df_forest["threshold"] == 30) &
    (df_forest["Year"] >= 2010)
].reset_index(drop = True)

df_forest["cum_forest_loss"] = df_forest.groupby(["Department", "Municipality"])["forest_loss_ha"].cumsum()
df_forest["Forest Loss %"] = df_forest["cum_forest_loss"] / df_forest["extent_2010_ha"]
df_forest = df_forest[[
    "Department", "Municipality", "Lat", "Long", "elevation", "Year", "Forest Loss %"
]]

df_forest

,Department,Municipality,Lat,Long,elevation,Year,Forest Loss %
0,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,2010,0.000339
1,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,2011,0.000531
2,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,2012,0.000729
3,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,2013,0.000872
4,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,2014,0.001111
...,...,...,...,...,...,...,...
13840,Vichada,Santa Rosalía,5.135381,-70.864961,116.421806,2018,0.005562
13841,Vichada,Santa Rosalía,5.135381,-70.864961,116.421806,2019,0.006177
13842,Vichada,Santa Rosalía,5.135381,-70.864961,116.421806,2020,0.007157
13843,Vichada,Santa Rosalía,5.135381,-70.864961,116.421806,2021,0.008935


# Landslides

In [8]:
df_landslides = pd.read_excel("../datasets/original/desinventar_landslides.xls")

display(df_landslides.info())
display(df_landslides.describe())
df_landslides

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10438 entries, 0 to 10437
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Serial                10438 non-null  object 
 1   Event                 10438 non-null  object 
 2   Date (YMD)            10438 non-null  object 
 3   Comments              7908 non-null   object 
 4   Cause                 10438 non-null  object 
 5   Description of Cause  1028 non-null   object 
 6   Source                10438 non-null  object 
 7   Magnitude             71 non-null     object 
 8   GLIDEnumber           0 non-null      float64
 9   Other sectors         1182 non-null   object 
 10  DataCards             10438 non-null  int64  
 11  Deaths                1556 non-null   float64
 12  Injured               881 non-null    float64
 13  Missing               166 non-null    float64
 14  Houses Destroyed      2249 non-null   float64
 15  Houses Damaged     

None

,GLIDEnumber,DataCards,Deaths,Injured,Missing,Houses Destroyed,Houses Damaged,Directly affected,Indirectly Affected,Relocated,Evacuated,Losses $USD,Losses $Local,Education centers,Hospitals,Damages in crops Ha.,Lost Cattle,Damages in roads Mts,Duration (d),fichas.latitude,fichas.longitude
count,0.0,10438.0,1556.000000,881.000000,166.000000,2249.000000,2615.000000,488.000000,3839.000000,11.000000,178.000000,4.000000e+00,2.050000e+02,291.000000,37.000000,237.000000,5.000000,747.000000,70.000000,0.0,0.0
mean,NaN,1.0,4.699871,4.396141,5.138554,10.668742,38.911663,139.747951,710.613701,282.727273,241.898876,4.100000e+06,1.964098e+08,2.134021,1.243243,1080.534346,30.200000,722.641232,8.128571,NaN,NaN
std,NaN,0.0,16.654014,8.539530,11.957453,39.050600,120.455401,575.937507,10786.405284,737.457265,630.320671,4.793050e+06,2.106452e+09,3.465979,1.038305,4676.609490,23.477649,6982.412451,19.241351,NaN,NaN
min,NaN,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000,5.000000e-01,2.000000e+01,1.000000,1.000000,0.020000,9.000000,1.000000,2.000000,NaN,NaN
25%,NaN,1.0,1.000000,1.000000,1.000000,1.000000,2.000000,6.000000,10.000000,16.000000,24.000000,3.000001e+05,6.850000e+05,1.000000,1.000000,20.000000,10.000000,1.000000,2.000000,NaN,NaN
50%,NaN,1.0,2.000000,2.000000,2.000000,2.000000,8.000000,25.000000,45.000000,65.000000,53.000000,3.200000e+06,3.839900e+06,1.000000,1.000000,80.000000,22.000000,1.000000,4.000000,NaN,NaN
75%,NaN,1.0,4.000000,4.000000,4.000000,7.000000,30.000000,75.000000,200.000000,119.000000,203.750000,7.000000e+06,1.000000e+07,2.000000,1.000000,311.000000,50.000000,5.000000,6.000000,NaN,NaN
max,NaN,1.0,500.000000,150.000000,100.000000,949.000000,2635.000000,8985.000000,400000.000000,2500.000000,5000.000000,1.000000e+07,3.000000e+10,50.000000,7.000000,40708.000000,60.000000,160000.000000,150.000000,NaN,NaN


,Serial,Event,Date (YMD),Comments,Cause,Description of Cause,Source,Magnitude,GLIDEnumber,Other sectors,DataCards,Deaths,Injured,Missing,Houses Destroyed,Houses Damaged,Directly affected,Indirectly Affected,Relocated,Evacuated,Losses $USD,Losses $Local,Education centers,Hospitals,Damages in crops Ha.,Lost Cattle,Damages in roads Mts,Duration (d),fichas.latitude,fichas.longitude,Departamento,Municipio
0,UNGRD-2016-0344,LANDSLIDE,2016/12/31,MUNICIPIO IQUIRA INFORMACIÓN PRELIMINAR EMERGE...,UNKNOWN,NaN,UNGRD,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Huila,Iquira
1,UNGRD-2016-0344,LANDSLIDE,2016/12/30,DCC INFORMA QUE EN EL MUNICIPIO MOCOA VÍA MOCO...,UNKNOWN,NaN,UNGRD,NaN,NaN,3 Vehículos,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Putumayo,Mocoa
2,UNGRD-2016-0343,LANDSLIDE,2016/12/28,CDGRD DE ANTIOQUIA; INFORMA; MUNICIPIO DE CO...,UNKNOWN,NaN,UNGRD,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,262.0,NaN,262.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Antioquia,Copacabana
3,UNGRD-2016-0343,LANDSLIDE,2016/12/27,CDGRD DEL CAUCA; REPORTA; MUNICIPIO DE TIBIO; ...,UNKNOWN,NaN,UNGRD,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cauca,Timbío
4,UNGRD-2016-0343,LANDSLIDE,2016/12/25,CDGRD DE CUNDINAMARCA; INFORMA; CARLOS MARIO C...,RAIN,NaN,UNGRD,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,13.0,NaN,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cundinamarca,Pandi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10433,1984-0470,LANDSLIDE,1984/10/25,En la intendencia del Putumayo en el sitio El...,RAIN,NaN,EL TIEMPO 1984-10-26,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Putumayo,Mocoa
10434,1984-0469,LANDSLIDE,1984/10/25,Sobre la carretera al mar el Distrito de Carr...,RAIN,NaN,EL TIEMPO 1984-10-26,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nariño,Ricaurte
10435,1984-0468,LANDSLIDE,1984/10/25,En la vía que de Pasto conduce al aeropuerto A...,RAIN,NaN,EL TIEMPO 1984-10-26,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nariño,Pasto
10436,1984-0467,LANDSLIDE,1984/10/25,Sobre la carretera al mar el Distrito de Carr...,RAIN,NaN,EL TIEMPO 1984-10-26,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nariño,Barbacoas


In [9]:
df_landslides["Date (YMD)"] = pd.to_datetime(df_landslides["Date (YMD)"], format = "%Y/%m/%d", errors = "coerce")
df_landslides["Year"] = df_landslides["Date (YMD)"].dt.year
df_landslides = df_landslides[df_landslides["Year"] >= 2010].sort_values("Date (YMD)")
df_landslides = df_landslides[["Date (YMD)", "Departamento", "Municipio", "Year"]].dropna().reset_index(drop = True)

display(df_landslides.info())
df_landslides["Year"].value_counts(sort = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3737 entries, 0 to 3736
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date (YMD)    3737 non-null   datetime64[ns]
 1   Departamento  3737 non-null   object        
 2   Municipio     3737 non-null   object        
 3   Year          3737 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 116.9+ KB


None

Year
2010.0    518
2011.0    873
2012.0    528
2013.0    421
2014.0    359
2015.0    225
2016.0    338
2017.0    475
Name: count, dtype: int64

In [10]:
departments = df_forest["Department"].unique()
municipalities = df_forest["Municipality"].unique()
departments.sort()
municipalities.sort()

dept_munic = {}
for dept in departments:
    df_tmp = df_forest[df_forest["Department"] == dept]
    dept_munic[dept] = df_tmp["Municipality"].unique()

In [11]:
pd.set_option('display.max_rows', None)

replacements = [
    ("Antioquia", "La Pintada", "Antioquia", "Abejorral"),
    ("Antioquia", "La Unión", "Antioquia", "La Unión de Sucre"),
    ("Bogotá D.C.", "Bogota", "Cundinamarca", "Santafé de Bogotá"),
    ("Bolívar", "Cartagena", "Bolívar", "Cartagena de Indias"),
    ("Caldas",	"Norcasia", "Caldas", "Samaná"),
    ("Cauca", "López", "Cauca", "López de Micay"),
    ("Cauca", "Piamonte", "Cauca", "Santa Rosa"),
    ("Cauca", "Villa Rica", "Cauca", "Buenos Aires"),
    ("Cesar", "Pueblo Bello", "Cesar", "Valledupar"),
    ("Chocó", "Atrato", "Chocó", "El Carmen de Atrato"),
    ("Chocó", "Medio Atrato", "Chocó", "Nuquí"),
    ("Chocó", "Medio Baudo", "Chocó", "Bajo Baudó"),
    ("Chocó", "Medio San Juan", "Chocó", "Quibdó"),
    ("Cundinamarca", "El Rosal", "Cundinamarca", "Facatativá"),
    ("La Guajira", "Albania", "La Guajira", "Barrancas"),
    ("Nariño", "El Peñol", "Nariño", "Los Andes"),
    ("Nariño", "Nariño", "Nariño", "La Florida"),
    ("Nariño", "Pasto", "Nariño", "San Juan de Pasto"),
    ("Norte de Santander", "Cúcuta", "Norte de Santander", "San José de Cúcuta"),
    ("Norte de Santander", "Salazar", "Norte de Santander", "Salazar de las Palmas"),
    ("Norte de Santander", "Silos", "Norte de Santander", "Santo Domingo de Silos"),
    ("Putumayo", "Mocoa", "Putumayo", "San Miguel de Mocoa"),
    ("Tolima", "Palocabildo", "Tolima", "Fresno"),
    ("Valle del Cauca", "Cali", "Valle del Cauca", "Calima")
]

df_landslides2 = df_landslides.copy()
for i, row in df_landslides.iterrows():
    dept = row["Departamento"]
    munic = row["Municipio"]
    
    found = False
    for replacement in replacements:
        if dept == replacement[0] and munic== replacement[1]:
            df_landslides2.at[i, "Departamento"] = replacement[2]
            df_landslides2.at[i, "Municipio"] = replacement[3]
            found = True
            break
        
    if not found:
        if dept not in departments:
            results = difflib.get_close_matches(dept, departments)
            if len(results) > 0 and dept_munic and munic in dept_munic[results[0]]:
                df_landslides2.at[i, "Departamento"] = results[0]
        elif munic not in dept_munic[dept]:
            results = difflib.get_close_matches(munic, dept_munic[dept])
            if len(results) > 0:
                df_landslides2.at[i, "Municipio"] = results[0]
            else:
                df_landslides2.at[i, "Municipio"] = "{}_".format(munic)
            
df_missing = df_landslides2.loc[
    (~df_landslides2["Municipio"].isin(municipalities)) |
    (~df_landslides2["Departamento"].isin(departments))
][["Departamento", "Municipio"]].drop_duplicates()
print(len(df_missing))
display(df_missing.sort_values(["Departamento", "Municipio"]))

pd.set_option('display.max_rows', 10)

0


,Departamento,Municipio


In [12]:
df_landslides = (
    df_landslides2
        .copy()
        .rename({ 
            "Date (YMD)": "Date",
            "Departamento": "Department",
            "Municipio": "Municipality"
        }, axis = 1)
        .drop("Year", axis = 1)
)
df_landslides["Landslide"] = 1

del df_landslides2
df_landslides

,Date,Department,Municipality,Landslide
0,2010-01-10,Cundinamarca,Santafé de Bogotá,1
1,2010-02-09,Boyacá,San Mateo,1
2,2010-02-11,Valle del Cauca,Buenaventura,1
3,2010-03-11,Caquetá,Belén de los Andaquies,1
4,2010-04-07,Nariño,Ricaurte,1
...,...,...,...,...
3732,2017-12-27,Cundinamarca,La Mesa,1
3733,2017-12-27,Quindío,Armenia,1
3734,2017-12-28,Nariño,Los Andes,1
3735,2017-12-29,Nariño,Buesaco,1


# Geographic Profile

In [13]:
soil_data = []
soil_path = "../datasets/original/soil"
soil_files = [ x for x in os.listdir(soil_path) if x.endswith(".csv") ]
for file in soil_files:
    soil_data.append(pd.read_csv(os.path.join(soil_path, file), delimiter=";"))
    
df_soil = pd.DataFrame()
for i, df in enumerate(soil_data):
    if i == 0:
        df_soil = df
    else:
        df_soil = pd.merge(df_soil, df, on = ["profile_identifier", "latitude", "longitude"])
       
df_soil.drop("profile_identifier", axis = 1, inplace = True)
df_soil.rename({
    "latitude": "Lat",
    "longitude": "Long"
}, axis = 1, inplace = True)

display(df_soil.info())
display(df_soil.describe())
df_soil

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3597 entries, 0 to 3596
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Lat                3597 non-null   float64
 1   Long               3597 non-null   float64
 2   clay.15.30.cm      3597 non-null   float64
 3   silt.15.30.cm      3597 non-null   float64
 4   sand.15.30.cm      3597 non-null   float64
 5   trans_1.15.30.cm   3597 non-null   float64
 6   trans_2.15.30.cm   3597 non-null   float64
 7   clay.30.60.cm      3597 non-null   float64
 8   silt.30.60.cm      3597 non-null   float64
 9   sand.30.60.cm      3597 non-null   float64
 10  trans_1.30.60.cm   3597 non-null   float64
 11  trans_2.30.60.cm   3597 non-null   float64
 12  clay.60.100.cm     3597 non-null   float64
 13  silt.60.100.cm     3597 non-null   float64
 14  sand.60.100.cm     3597 non-null   float64
 15  trans_1.60.100.cm  3597 non-null   float64
 16  trans_2.60.100.cm  3597 

None

,Lat,Long,clay.15.30.cm,silt.15.30.cm,sand.15.30.cm,trans_1.15.30.cm,trans_2.15.30.cm,clay.30.60.cm,silt.30.60.cm,sand.30.60.cm,trans_1.30.60.cm,trans_2.30.60.cm,clay.60.100.cm,silt.60.100.cm,sand.60.100.cm,trans_1.60.100.cm,trans_2.60.100.cm,clay.0.5.cm,silt.0.5.cm,sand.0.5.cm,trans_1.0.5.cm,trans_2.0.5.cm,clay.5.15.cm,silt.5.15.cm,sand.5.15.cm,trans_1.5.15.cm,trans_2.5.15.cm
count,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000,3597.000000
mean,5.094969,-74.535983,31.134112,29.673172,39.192994,0.195055,0.032441,32.788657,29.038504,38.172199,0.108532,-0.026477,32.758048,28.671754,38.571143,0.128082,-0.047445,28.429525,30.079900,41.491854,0.380428,0.145093,29.253489,29.997581,40.748457,0.317642,0.106027
std,3.238751,2.245669,17.818131,13.919947,22.491816,1.458507,0.837035,19.181745,14.695547,23.524036,1.612571,0.925598,19.696264,15.394645,24.379789,1.640493,0.944461,17.377104,14.097276,22.487443,1.458135,0.891015,17.185802,13.776060,22.169898,1.408969,0.823347
min,-4.157500,-78.930311,0.900000,0.400000,0.000000,-6.826588,-3.842309,0.000000,0.100000,0.000000,-7.473707,-4.666199,0.100000,0.100000,0.000000,-8.431450,-4.252908,0.000000,0.100000,0.100000,-6.061651,-6.332315,0.400000,1.000000,0.100000,-6.423241,-3.445800
25%,3.070182,-76.261740,17.500000,19.800000,20.500000,-0.704338,-0.486689,17.500000,18.400000,18.500000,-0.865292,-0.601655,16.700000,17.800000,18.300000,-0.893277,-0.634244,14.900000,20.000000,23.400000,-0.507658,-0.340205,16.000000,20.000000,22.900000,-0.547813,-0.364854
50%,4.829722,-75.069444,29.300000,28.000000,36.800000,0.205838,0.038327,30.900000,26.700000,35.100000,0.089326,-0.052321,30.700000,26.200000,34.800000,0.084991,-0.072895,25.800000,28.800000,40.800000,0.445728,0.151125,26.800000,28.800000,39.800000,0.377035,0.112739
75%,7.539351,-73.150560,42.400000,38.300000,56.200000,1.129975,0.556402,46.200000,38.100000,55.700000,1.082811,0.515046,47.400000,37.700000,56.500000,1.151132,0.503406,38.900000,39.300000,58.300000,1.328973,0.655968,39.800000,39.000000,57.100000,1.236136,0.606764
max,12.365633,-67.490242,94.800000,76.800000,98.000000,4.584967,3.328554,94.500000,90.100000,99.900000,9.143774,6.777700,97.100000,87.500000,99.800000,7.349827,4.191331,95.100000,76.500000,99.200000,7.678931,7.868013,94.500000,75.500000,98.000000,4.626233,4.820212


,Lat,Long,clay.15.30.cm,silt.15.30.cm,sand.15.30.cm,trans_1.15.30.cm,trans_2.15.30.cm,clay.30.60.cm,silt.30.60.cm,sand.30.60.cm,trans_1.30.60.cm,trans_2.30.60.cm,clay.60.100.cm,silt.60.100.cm,sand.60.100.cm,trans_1.60.100.cm,trans_2.60.100.cm,clay.0.5.cm,silt.0.5.cm,sand.0.5.cm,trans_1.0.5.cm,trans_2.0.5.cm,clay.5.15.cm,silt.5.15.cm,sand.5.15.cm,trans_1.5.15.cm,trans_2.5.15.cm
0,-0.570000,-72.470001,7.7,4.8,87.5,2.434181,-0.460365,10.1,3.8,86.1,2.142836,-0.983610,7.5,4.1,88.4,2.460745,-0.618121,3.0,6.3,90.7,3.417662,0.753923,4.0,6.0,90.0,3.110619,0.402068
1,-0.580000,-72.379997,40.5,41.6,17.8,-0.820955,0.026863,46.4,37.0,16.6,-1.028472,-0.227161,52.8,23.9,23.3,-0.816138,-0.793193,16.3,38.3,45.4,1.021779,0.853415,28.5,39.5,32.0,0.115188,0.325509
2,-1.280000,-70.830002,32.8,22.4,44.8,0.311428,-0.382280,35.3,20.3,44.4,0.231140,-0.553296,37.0,20.4,42.6,0.141165,-0.594406,13.1,20.4,66.4,1.621965,0.443425,22.6,21.7,55.8,0.904447,-0.041690
3,1.262778,-77.289722,36.4,34.6,29.1,-0.223105,-0.050356,37.5,33.6,28.9,-0.257821,-0.108507,37.9,34.7,27.4,-0.323111,-0.087352,36.0,34.9,29.1,-0.211244,-0.030793,36.1,34.8,29.1,-0.213628,-0.034713
4,1.116111,-77.427500,23.0,33.7,43.3,0.635890,0.384150,43.4,18.5,38.1,-0.131468,-0.855878,40.5,15.5,44.0,0.083357,-0.961414,22.3,31.4,46.3,0.731428,0.342901,21.8,32.5,45.7,0.742896,0.401350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3592,5.472748,-68.662856,4.0,29.9,66.1,2.805078,2.011114,4.0,24.6,71.4,2.881503,1.817904,4.0,20.6,75.4,2.936987,1.637259,4.0,31.9,64.1,2.774243,2.076124,4.0,31.5,64.5,2.780497,2.063436
3593,5.554294,-68.486588,18.4,62.0,19.5,0.059594,1.215032,36.8,49.8,13.4,-1.010922,0.303363,53.8,36.6,9.6,-1.719342,-0.385149,5.4,69.4,25.1,1.529909,2.546794,8.3,67.9,23.9,1.063263,2.107207
3594,5.574345,-68.560311,10.3,37.4,52.3,1.620716,1.284612,8.3,27.0,64.7,2.048444,1.173508,6.1,18.7,75.2,2.517121,1.126166,9.9,43.5,46.6,1.553065,1.484115,10.0,42.2,47.8,1.566136,1.443118
3595,4.466251,-67.876136,3.4,13.1,83.6,3.208091,1.351306,5.1,13.3,81.5,2.766057,0.956125,6.1,9.6,84.3,2.628635,0.453015,1.9,5.2,92.9,3.905810,1.025654,2.3,7.6,90.1,3.682131,1.210856


In [14]:
coords_rainfall = df_rainfall[["Lat", "Long"]].drop_duplicates()
coords_soil = df_soil[["Lat", "Long"]].drop_duplicates()

distances = distance_matrix(coords_rainfall, coords_soil)

coords_soil["Closest"] = np.argmin(distances, axis = 0)
coords_rainfall.reset_index(inplace = True, names = "Closest")
coords_soil = pd.merge(coords_soil, coords_rainfall, on = "Closest")

coords_soil

,Lat_x,Long_x,Closest,Lat_y,Long_y
0,-0.570000,-72.470001,6,-1.007618,-72.141613
1,-0.580000,-72.379997,6,-1.007618,-72.141613
2,-0.400000,-72.150000,6,-1.007618,-72.141613
3,-0.466667,-72.216667,6,-1.007618,-72.141613
4,-0.466667,-72.233333,6,-1.007618,-72.141613
...,...,...,...,...,...
3590,5.106861,-70.543333,449,5.135381,-70.864961
3591,4.899972,-71.007889,449,5.135381,-70.864961
3592,4.911889,-70.996389,449,5.135381,-70.864961
3593,5.106333,-70.829389,449,5.135381,-70.864961


In [46]:
df_soil2 = (
    pd.merge(
        df_soil,
        coords_soil,
        left_on = ["Lat", "Long"],
        right_on = ["Lat_x", "Long_x"]
    )
    .drop(["Lat", "Long", "Lat_x", "Long_x", "Closest"], axis = 1)
    .rename({
        "Lat_y": "Lat",
        "Long_y": "Long"
    }, axis = 1)
    .groupby(["Lat", "Long"])
    .mean()
    .reset_index()
    .merge(
        coords_rainfall.drop("Closest", axis = 1),
        how = "outer",
        on = ["Lat", "Long"]
    )
)

display(df_soil2.info())
display(df_soil2.describe())
df_soil2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063 entries, 0 to 1062
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Lat                1063 non-null   float64
 1   Long               1063 non-null   float64
 2   clay.15.30.cm      752 non-null    float64
 3   silt.15.30.cm      752 non-null    float64
 4   sand.15.30.cm      752 non-null    float64
 5   trans_1.15.30.cm   752 non-null    float64
 6   trans_2.15.30.cm   752 non-null    float64
 7   clay.30.60.cm      752 non-null    float64
 8   silt.30.60.cm      752 non-null    float64
 9   sand.30.60.cm      752 non-null    float64
 10  trans_1.30.60.cm   752 non-null    float64
 11  trans_2.30.60.cm   752 non-null    float64
 12  clay.60.100.cm     752 non-null    float64
 13  silt.60.100.cm     752 non-null    float64
 14  sand.60.100.cm     752 non-null    float64
 15  trans_1.60.100.cm  752 non-null    float64
 16  trans_2.60.100.cm  752 n

None

,Lat,Long,clay.15.30.cm,silt.15.30.cm,sand.15.30.cm,trans_1.15.30.cm,trans_2.15.30.cm,clay.30.60.cm,silt.30.60.cm,sand.30.60.cm,trans_1.30.60.cm,trans_2.30.60.cm,clay.60.100.cm,silt.60.100.cm,sand.60.100.cm,trans_1.60.100.cm,trans_2.60.100.cm,clay.0.5.cm,silt.0.5.cm,sand.0.5.cm,trans_1.0.5.cm,trans_2.0.5.cm,clay.5.15.cm,silt.5.15.cm,sand.5.15.cm,trans_1.5.15.cm,trans_2.5.15.cm
count,1063.000000,1063.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000
mean,5.545912,-74.628873,30.045788,28.718323,41.237050,0.312232,0.036984,31.748616,28.108253,40.141729,0.238781,-0.018780,32.088363,27.756717,40.155998,0.225115,-0.049557,27.736027,28.945100,43.318826,0.466421,0.125449,28.418108,28.948769,42.632173,0.414839,0.096948
std,2.573251,1.705331,12.538105,9.489505,16.610346,1.070776,0.598741,13.961537,10.093984,17.531299,1.174749,0.694286,14.647658,10.409927,18.335976,1.237828,0.701311,12.228128,9.412084,16.194141,1.038844,0.616880,12.104626,9.199312,16.101977,1.009488,0.580287
min,-4.203165,-81.718525,1.000000,1.000000,0.300000,-5.495522,-1.812175,1.000000,1.000000,0.200000,-6.031238,-1.977281,1.000000,1.000000,0.100000,-6.752452,-2.721824,1.000000,1.000000,0.100000,-5.851629,-2.150244,1.000000,1.000000,0.400000,-5.241948,-1.910537
25%,4.214098,-75.763132,20.996875,22.600000,29.300000,-0.321422,-0.337036,22.477273,21.452143,26.782031,-0.459069,-0.445525,22.075000,20.708333,26.412500,-0.481064,-0.471842,19.275000,22.831818,32.445833,-0.115074,-0.243252,19.847500,22.755357,31.885000,-0.161031,-0.261120
50%,5.500707,-74.753281,30.000000,28.074638,40.612500,0.304799,0.008055,31.650000,27.143182,38.950000,0.159692,-0.049099,31.075000,26.570833,38.942222,0.196998,-0.082942,26.335417,29.125000,44.257500,0.533498,0.114780,27.654615,29.083333,43.480000,0.457026,0.083164
75%,6.846930,-73.415627,37.400000,34.125000,52.600000,0.933066,0.346184,40.335714,33.451705,51.668750,0.884713,0.319805,41.644318,33.844394,52.315625,0.920783,0.308477,34.500000,34.500000,53.937500,1.077828,0.453506,35.037500,34.170000,53.606250,1.032385,0.423632
max,13.381425,-67.046459,70.150000,73.100000,98.000000,4.584967,2.693356,79.500000,74.200000,98.000000,4.584967,3.604661,90.000000,76.700000,98.000000,4.584967,3.533240,69.100000,73.800000,98.000000,4.584967,2.388898,67.128571,73.800000,98.000000,4.584967,1.906709


,Lat,Long,clay.15.30.cm,silt.15.30.cm,sand.15.30.cm,trans_1.15.30.cm,trans_2.15.30.cm,clay.30.60.cm,silt.30.60.cm,sand.30.60.cm,trans_1.30.60.cm,trans_2.30.60.cm,clay.60.100.cm,silt.60.100.cm,sand.60.100.cm,trans_1.60.100.cm,trans_2.60.100.cm,clay.0.5.cm,silt.0.5.cm,sand.0.5.cm,trans_1.0.5.cm,trans_2.0.5.cm,clay.5.15.cm,silt.5.15.cm,sand.5.15.cm,trans_1.5.15.cm,trans_2.5.15.cm
0,-4.203165,-69.935907,29.842857,29.142857,41.021429,0.269400,-0.019669,34.242857,27.628571,38.135714,0.041860,-0.216561,35.328571,25.971429,38.714286,0.094846,-0.212319,25.157143,31.957143,42.921429,0.501746,0.278671,26.342857,31.007143,42.664286,0.469199,0.188887
1,-3.788867,-70.355642,45.644444,35.933333,18.388889,-1.025749,-0.273298,52.600000,32.044444,15.322222,-1.463168,-0.544135,55.100000,28.955556,15.988889,-1.556529,-0.655293,38.111111,38.711111,23.155556,-0.534597,0.015020,39.900000,38.133333,21.944444,-0.656235,-0.056236
2,-2.458989,-69.859741,32.748571,39.374286,27.865714,-0.312876,0.283578,36.560000,36.965714,26.474286,-0.491475,0.126377,39.054286,35.108571,25.840000,-0.520405,0.037771,30.342857,39.988571,29.671429,-0.155831,0.395876,30.825714,39.997143,29.185714,-0.193428,0.369937
3,-1.747145,-73.209101,34.850000,21.550000,43.583333,0.303875,-0.393698,42.466667,18.550000,39.000000,0.011093,-0.759940,44.733333,21.683333,33.583333,-0.217337,-0.770324,26.133333,24.966667,48.916667,0.690841,0.008691,28.550000,24.166667,47.266667,0.577684,-0.097297
4,-1.378972,-72.804280,27.706667,33.246667,39.033333,0.299123,0.296019,32.280000,30.853333,36.886667,-0.005816,0.017853,37.960000,28.533333,33.493333,-0.375715,-0.248179,25.253333,33.640000,41.086667,0.499962,0.310814,25.773333,33.633333,40.600000,0.444680,0.350215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,10.777471,-74.853162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1059,10.917119,-74.799151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1060,12.556732,-81.718525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1061,12.576855,-81.705052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
imputer = KNNImputer(n_neighbors = 3)
cols = list(df_soil2.columns)
df_soil2[cols] = imputer.fit_transform(df_soil2[cols])

display(df_soil2.info())
display(df_soil2.describe())
df_soil2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063 entries, 0 to 1062
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Lat                1063 non-null   float64
 1   Long               1063 non-null   float64
 2   clay.15.30.cm      1063 non-null   float64
 3   silt.15.30.cm      1063 non-null   float64
 4   sand.15.30.cm      1063 non-null   float64
 5   trans_1.15.30.cm   1063 non-null   float64
 6   trans_2.15.30.cm   1063 non-null   float64
 7   clay.30.60.cm      1063 non-null   float64
 8   silt.30.60.cm      1063 non-null   float64
 9   sand.30.60.cm      1063 non-null   float64
 10  trans_1.30.60.cm   1063 non-null   float64
 11  trans_2.30.60.cm   1063 non-null   float64
 12  clay.60.100.cm     1063 non-null   float64
 13  silt.60.100.cm     1063 non-null   float64
 14  sand.60.100.cm     1063 non-null   float64
 15  trans_1.60.100.cm  1063 non-null   float64
 16  trans_2.60.100.cm  1063 

None

,Lat,Long,clay.15.30.cm,silt.15.30.cm,sand.15.30.cm,trans_1.15.30.cm,trans_2.15.30.cm,clay.30.60.cm,silt.30.60.cm,sand.30.60.cm,trans_1.30.60.cm,trans_2.30.60.cm,clay.60.100.cm,silt.60.100.cm,sand.60.100.cm,trans_1.60.100.cm,trans_2.60.100.cm,clay.0.5.cm,silt.0.5.cm,sand.0.5.cm,trans_1.0.5.cm,trans_2.0.5.cm,clay.5.15.cm,silt.5.15.cm,sand.5.15.cm,trans_1.5.15.cm,trans_2.5.15.cm
count,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000
mean,5.545912,-74.628873,30.088215,28.932578,40.979052,0.286787,0.038594,31.606602,28.225727,40.166450,0.234258,-0.014053,31.873653,27.860881,40.266452,0.227881,-0.040610,27.898909,29.205566,42.895299,0.436412,0.120732,28.571492,29.219610,42.207362,0.386634,0.095310
std,2.573251,1.705331,11.602822,8.838715,15.588440,1.018250,0.548231,12.883671,9.348491,16.514592,1.101379,0.631265,13.508364,9.657882,17.190589,1.164997,0.645574,11.336336,8.871700,15.234527,0.977077,0.568047,11.248466,8.664641,15.168535,0.954319,0.536062
min,-4.203165,-81.718525,1.000000,1.000000,0.300000,-5.495522,-1.812175,1.000000,1.000000,0.200000,-6.031238,-1.977281,1.000000,1.000000,0.100000,-6.752452,-2.721824,1.000000,1.000000,0.100000,-5.851629,-2.150244,1.000000,1.000000,0.400000,-5.241948,-1.910537
25%,4.214098,-75.763132,22.156818,23.312500,29.563194,-0.316303,-0.297884,23.275000,22.200000,27.625000,-0.434092,-0.403518,23.104167,21.219643,27.933333,-0.428015,-0.414114,19.987500,23.700000,32.475000,-0.136105,-0.204833,20.690909,23.900000,31.800000,-0.174354,-0.222010
50%,5.500707,-74.753281,30.100000,28.600000,41.172727,0.322354,0.008910,31.825000,27.266667,39.350000,0.193952,-0.049763,31.282143,26.952381,39.134524,0.225972,-0.069063,26.866667,29.300000,43.996410,0.524336,0.115900,27.900000,29.366667,43.330000,0.453834,0.086704
75%,6.846930,-73.415627,37.113889,33.616667,51.483333,0.859447,0.320602,39.258333,33.293333,50.534091,0.840355,0.294154,40.387879,33.575000,51.400000,0.879245,0.274027,34.581667,34.325000,52.966667,1.011150,0.430297,35.281944,34.052778,52.500000,0.950703,0.400579
max,13.381425,-67.046459,70.150000,73.100000,98.000000,4.584967,2.693356,79.500000,74.200000,98.000000,4.584967,3.604661,90.000000,76.700000,98.000000,4.584967,3.533240,69.100000,73.800000,98.000000,4.584967,2.388898,67.128571,73.800000,98.000000,4.584967,1.906709


,Lat,Long,clay.15.30.cm,silt.15.30.cm,sand.15.30.cm,trans_1.15.30.cm,trans_2.15.30.cm,clay.30.60.cm,silt.30.60.cm,sand.30.60.cm,trans_1.30.60.cm,trans_2.30.60.cm,clay.60.100.cm,silt.60.100.cm,sand.60.100.cm,trans_1.60.100.cm,trans_2.60.100.cm,clay.0.5.cm,silt.0.5.cm,sand.0.5.cm,trans_1.0.5.cm,trans_2.0.5.cm,clay.5.15.cm,silt.5.15.cm,sand.5.15.cm,trans_1.5.15.cm,trans_2.5.15.cm
0,-4.203165,-69.935907,29.842857,29.142857,41.021429,0.269400,-0.019669,34.242857,27.628571,38.135714,0.041860,-0.216561,35.328571,25.971429,38.714286,0.094846,-0.212319,25.157143,31.957143,42.921429,0.501746,0.278671,26.342857,31.007143,42.664286,0.469199,0.188887
1,-3.788867,-70.355642,45.644444,35.933333,18.388889,-1.025749,-0.273298,52.600000,32.044444,15.322222,-1.463168,-0.544135,55.100000,28.955556,15.988889,-1.556529,-0.655293,38.111111,38.711111,23.155556,-0.534597,0.015020,39.900000,38.133333,21.944444,-0.656235,-0.056236
2,-2.458989,-69.859741,32.748571,39.374286,27.865714,-0.312876,0.283578,36.560000,36.965714,26.474286,-0.491475,0.126377,39.054286,35.108571,25.840000,-0.520405,0.037771,30.342857,39.988571,29.671429,-0.155831,0.395876,30.825714,39.997143,29.185714,-0.193428,0.369937
3,-1.747145,-73.209101,34.850000,21.550000,43.583333,0.303875,-0.393698,42.466667,18.550000,39.000000,0.011093,-0.759940,44.733333,21.683333,33.583333,-0.217337,-0.770324,26.133333,24.966667,48.916667,0.690841,0.008691,28.550000,24.166667,47.266667,0.577684,-0.097297
4,-1.378972,-72.804280,27.706667,33.246667,39.033333,0.299123,0.296019,32.280000,30.853333,36.886667,-0.005816,0.017853,37.960000,28.533333,33.493333,-0.375715,-0.248179,25.253333,33.640000,41.086667,0.499962,0.310814,25.773333,33.633333,40.600000,0.444680,0.350215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,10.777471,-74.853162,28.044048,29.715476,42.222619,0.172137,0.148276,27.159524,30.529762,42.302381,0.476891,0.096386,25.830952,32.517857,41.638095,0.518271,0.241884,25.073810,29.638095,45.276190,0.737211,0.382708,25.686905,30.291667,44.013095,0.603478,0.323315
1059,10.917119,-74.799151,35.563889,16.205556,48.219444,0.403848,-0.568096,37.500000,16.269444,46.230556,0.335699,-0.550565,35.175000,18.413889,46.408333,0.477780,-0.414177,32.461111,16.661111,50.888889,0.535316,-0.532713,33.361111,16.497222,50.141667,0.579808,-0.546089
1060,12.556732,-81.718525,33.840064,26.011645,40.156410,0.174920,-0.186898,33.991026,25.305128,40.709402,0.245960,-0.262209,31.649466,26.483440,41.843483,0.330037,-0.231230,29.765919,27.031944,43.212821,0.385244,-0.151108,31.610897,26.579808,41.804380,0.301403,-0.173633
1061,12.576855,-81.705052,33.840064,26.011645,40.156410,0.174920,-0.186898,33.991026,25.305128,40.709402,0.245960,-0.262209,31.649466,26.483440,41.843483,0.330037,-0.231230,29.765919,27.031944,43.212821,0.385244,-0.151108,31.610897,26.579808,41.804380,0.301403,-0.173633


# Merge All Datasets

In [48]:
df_rainfall["Date"] = pd.to_datetime(df_rainfall["Date"], format = "%Y-%m-%d")
df_rainfall["Year"] = df_rainfall["Date"].dt.year

df = pd.merge(df_rainfall, df_forest, how = "inner", on = ["Lat", "Long", "Year"])
df = pd.merge(df, df_landslides, "outer", ["Department", "Municipality", "Date"])
df = pd.merge(df, df_soil2, how = "outer", on = ["Lat", "Long"])
df["Landslide"] = df["Landslide"].fillna(0).astype(int)

display(df.info())
display(df.describe())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3366780 entries, 0 to 3366779
Data columns (total 36 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Date               datetime64[ns]
 1   Lat                float64       
 2   Long               float64       
 3   Total Rainfall     float64       
 4   Rainfall Duration  int64         
 5   Year               int32         
 6   Department         object        
 7   Municipality       object        
 8   elevation          float64       
 9   Forest Loss %      float64       
 10  Landslide          int64         
 11  clay.15.30.cm      float64       
 12  silt.15.30.cm      float64       
 13  sand.15.30.cm      float64       
 14  trans_1.15.30.cm   float64       
 15  trans_2.15.30.cm   float64       
 16  clay.30.60.cm      float64       
 17  silt.30.60.cm      float64       
 18  sand.30.60.cm      float64       
 19  trans_1.30.60.cm   float64       
 20  trans_2.30.60.cm   float

None

,Date,Lat,Long,Total Rainfall,Rainfall Duration,Year,elevation,Forest Loss %,Landslide,clay.15.30.cm,silt.15.30.cm,sand.15.30.cm,trans_1.15.30.cm,trans_2.15.30.cm,clay.30.60.cm,silt.30.60.cm,sand.30.60.cm,trans_1.30.60.cm,trans_2.30.60.cm,clay.60.100.cm,silt.60.100.cm,sand.60.100.cm,trans_1.60.100.cm,trans_2.60.100.cm,clay.0.5.cm,silt.0.5.cm,sand.0.5.cm,trans_1.0.5.cm,trans_2.0.5.cm,clay.5.15.cm,silt.5.15.cm,sand.5.15.cm,trans_1.5.15.cm,trans_2.5.15.cm
count,3366780,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.363858e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06,3.366780e+06
mean,2014-05-04 20:44:41.064279808,5.573825e+00,-7.462040e+01,6.243005e+00,1.044881e+01,2.013841e+03,1.187612e+03,1.575136e-02,1.109963e-03,3.020432e+01,2.897525e+01,4.082026e+01,2.769118e-01,3.621447e-02,3.175217e+01,2.823785e+01,4.000874e+01,2.236350e-01,-1.790059e-02,3.202959e+01,2.786912e+01,4.010222e+01,2.156037e-01,-4.491383e-02,2.796620e+01,2.928089e+01,4.275261e+01,4.287627e-01,1.219548e-01,2.864972e+01,2.928614e+01,4.206267e+01,3.783149e-01,9.528678e-02
min,2010-01-01 00:00:00,-4.203165e+00,-8.171853e+01,0.000000e+00,0.000000e+00,2.010000e+03,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,3.000000e-01,-5.495522e+00,-1.812175e+00,1.000000e+00,1.000000e+00,2.000000e-01,-6.031238e+00,-1.977281e+00,1.000000e+00,1.000000e+00,1.000000e-01,-6.752452e+00,-2.721824e+00,1.000000e+00,1.000000e+00,1.000000e-01,-5.851629e+00,-2.150244e+00,1.000000e+00,1.000000e+00,4.000000e-01,-5.241948e+00,-1.910537e+00
25%,2012-03-01 00:00:00,4.271262e+00,-7.574878e+01,4.000000e-01,2.000000e+00,2.012000e+03,2.170473e+02,3.815580e-03,0.000000e+00,2.230000e+01,2.337742e+01,2.950000e+01,-3.228030e-01,-2.985372e-01,2.346667e+01,2.222222e+01,2.752500e+01,-4.371720e-01,-4.072478e-01,2.315000e+01,2.124000e+01,2.788611e+01,-4.319710e-01,-4.150082e-01,2.000000e+01,2.375000e+01,3.225000e+01,-1.501217e-01,-2.036519e-01,2.070000e+01,2.395000e+01,3.175278e+01,-1.815033e-01,-2.196060e-01
50%,2014-04-30 00:00:00,5.518121e+00,-7.474952e+01,2.800000e+00,1.000000e+01,2.014000e+03,1.191456e+03,9.331614e-03,0.000000e+00,3.020000e+01,2.862500e+01,4.100000e+01,3.134099e-01,7.787500e-03,3.186667e+01,2.726667e+01,3.921667e+01,1.832900e-01,-5.219308e-02,3.154000e+01,2.695238e+01,3.910000e+01,2.116875e-01,-6.956244e-02,2.690000e+01,2.933810e+01,4.394286e+01,5.219970e-01,1.163326e-01,2.812000e+01,2.947167e+01,4.330000e+01,4.476515e-01,8.670383e-02
75%,2016-06-28 00:00:00,6.878518e+00,-7.342148e+01,8.300000e+00,1.700000e+01,2.016000e+03,1.915797e+03,2.023926e-02,0.000000e+00,3.728333e+01,3.361667e+01,5.131250e+01,8.464378e-01,3.169904e-01,3.953333e+01,3.326667e+01,5.033333e+01,8.326655e-01,2.889825e-01,4.063333e+01,3.355000e+01,5.115000e+01,8.741510e-01,2.673168e-01,3.464444e+01,3.436667e+01,5.280909e+01,1.007845e+00,4.312160e-01,3.533750e+01,3.410000e+01,5.210000e+01,9.411522e-01,4.005099e-01
max,2018-12-31 00:00:00,1.338142e+01,-6.704646e+01,3.723000e+02,2.400000e+01,2.018000e+03,3.670899e+03,1.782083e-01,1.000000e+00,7.015000e+01,7.310000e+01,9.800000e+01,4.584967e+00,2.693356e+00,7.950000e+01,7.420000e+01,9.800000e+01,4.584967e+00,3.604661e+00,9.000000e+01,7.670000e+01,9.800000e+01,4.584967e+00,3.533240e+00,6.910000e+01,7.380000e+01,9.800000e+01,4.584967e+00,2.388898e+00,6.712857e+01,7.380000e+01,9.800000e+01,4.584967e+00,1.906709e+00
std,NaN,2.569085e+00,1.689264e+00,9.998163e+00,8.416881e+00,2.503658e+00,9.354618e+02,1.879648e-02,3.329761e-02,1.161183e+01,8.827340e+00,1.555285e+01,1.017240e+00,5.467555e-01,1.289497e+01,9.315831e+00,1.645782e+01,1.101695e+00,6.316012e-01,1.353755e+01,9.620686e+00,1.714546e+01,1.166578e+00,6.459796e-01,1.136103e+01,8.856260e+00,1.52230

,Date,Lat,Long,Total Rainfall,Rainfall Duration,Year,Department,Municipality,elevation,Forest Loss %,Landslide,clay.15.30.cm,silt.15.30.cm,sand.15.30.cm,trans_1.15.30.cm,trans_2.15.30.cm,clay.30.60.cm,silt.30.60.cm,sand.30.60.cm,trans_1.30.60.cm,trans_2.30.60.cm,clay.60.100.cm,silt.60.100.cm,sand.60.100.cm,trans_1.60.100.cm,trans_2.60.100.cm,clay.0.5.cm,silt.0.5.cm,sand.0.5.cm,trans_1.0.5.cm,trans_2.0.5.cm,clay.5.15.cm,silt.5.15.cm,sand.5.15.cm,trans_1.5.15.cm,trans_2.5.15.cm
0,2010-01-01,-4.203165,-69.935907,0.4,4,2010,Amazonas,Leticia,78.378479,0.000479,0,29.842857,29.142857,41.021429,0.269400,-0.019669,34.242857,27.628571,38.135714,0.041860,-0.216561,35.328571,25.971429,38.714286,0.094846,-0.212319,25.157143,31.957143,42.921429,0.501746,0.278671,26.342857,31.007143,42.664286,0.469199,0.188887
1,2010-01-02,-4.203165,-69.935907,0.0,0,2010,Amazonas,Leticia,78.378479,0.000479,0,29.842857,29.142857,41.021429,0.269400,-0.019669,34.242857,27.628571,38.135714,0.041860,-0.216561,35.328571,25.971429,38.714286,0.094846,-0.212319,25.157143,31.957143,42.921429,0.501746,0.278671,26.342857,31.007143,42.664286,0.469199,0.188887
2,2010-01-03,-4.203165,-69.935907,1.6,11,2010,Amazonas,Leticia,78.378479,0.000479,0,29.842857,29.142857,41.021429,0.269400,-0.019669,34.242857,27.628571,38.135714,0.041860,-0.216561,35.328571,25.971429,38.714286,0.094846,-0.212319,25.157143,31.957143,42.921429,0.501746,0.278671,26.342857,31.007143,42.664286,0.469199,0.188887
3,2010-01-04,-4.203165,-69.935907,6.7,17,2010,Amazonas,Leticia,78.378479,0.000479,0,29.842857,29.142857,41.021429,0.269400,-0.019669,34.242857,27.628571,38.135714,0.041860,-0.216561,35.328571,25.971429,38.714286,0.094846,-0.212319,25.157143,31.957143,42.921429,0.501746,0.278671,26.342857,31.007143,42.664286,0.469199,0.188887
4,2010-01-05,-4.203165,-69.935907,24.9,24,2010,Amazonas,Leticia,78.378479,0.000479,0,29.842857,29.142857,41.021429,0.269400,-0.019669,34.242857,27.628571,38.135714,0.041860,-0.216561,35.328571,25.971429,38.714286,0.094846,-0.212319,25.157143,31.957143,42.921429,0.501746,0.278671,26.342857,31.007143,42.664286,0.469199,0.188887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3366775,2017-12-27,13.381425,-81.365743,1.6,13,2017,San Andrés y Providencia,Providencia,22.483635,0.000564,0,39.882564,37.316923,22.803077,-0.740396,-0.082825,41.234359,37.485128,21.287179,-0.844785,-0.110689,39.069744,37.908718,23.021538,-0.645845,-0.021011,33.865641,36.380000,29.759487,-0.248561,0.080606,35.962564,36.572308,27.457436,-0.398335,0.017399
3366776,2017-12-28,13.381425,-81.365743,6.2,23,2017,San Andrés y Providencia,Providencia,22.483635,0.000564,0,39.882564,37.316923,22.803077,-0.740396,-0.082825,41.234359,37.485128,21.287179,-0.844785,-0.110689,39.069744,37.908718,23.021538,-0.645845,-0.021011,33.865641,36.380000,29.759487,-0.248561,0.080606,35.962564,36.572308,27.457436,-0.398335,0.017399
3366777,2017-12-29,13.381425,-81.365743,9.9,23,2017,San Andrés y Providencia,Providencia,22.483635,0.000564,0,39.882564,37.316923,22.803077,-0.740396,-0.082825,41.234359,37.485128,21.287179,-0.844785,-0.110689,39.069744,37.908718,23.021538,-0.645845,-0.021011,33.865641,36.380000,29.759487,-0.248561,0.080606,35.962564,36.572308,27.457436,-0.398335,0.017399
3366778,2017-12-30,13.381425,-81.365743,2.6,17,2017,San Andrés y Providencia,Providencia,22.483635,0.000564,0,39.882564,37.316923,22.803077,-0.740396,-0.082825,41.234359,37.485128,21.287179,-0.844785,-0.110689,39.069744,37.908718,23.021538,-0.645845,-0.021011,33.865641,36.380000,29.759487,-0.248561,0.080606,35.962564,36.572308,27.457436,-0.398335,0.017399


In [49]:
df.to_csv("../datasets/full_dataset.csv")